In [1]:
alphabet = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y'] 
masses = [71, 103, 115, 129, 147, 57, 137, 113, 128, 113, 131, 114, 97, 128, 156, 87, 101, 99, 186, 163]
weights = dict(zip(alphabet, masses))

def ideal_spectrum(peptide):
    res = []
    tmp_l, tmp_r = 0, 0
    res.append(0)
    for i in range(len(peptide)):
        tmp_l += weights[peptide[i]]
        res.append(tmp_l)
        tmp_r += weights[peptide[len(peptide) - 1 - i]]
        res.append(tmp_r)
    res = list(set(res))
    res.sort()
    print(' '.join(map(str, res)))
    
import sys

def main():
    sys.stdin = open("test_in.txt", "r")
    sys.stdout = open("test_out.txt", "w")
    input = sys.stdin.readline
    
    peptide = input().rstrip('\n')
    ideal_spectrum(peptide)
main()

In [59]:
alphabet = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y'] 
masses = [71, 103, 115, 129, 147, 57, 137, 113, 128, 113, 131, 114, 97, 128, 156, 87, 101, 99, 186, 163]
weights = dict(zip(alphabet, masses))
acid_from_mass = dict(zip(masses, alphabet))

spectrum = []
edges = []
possible_paths = []
path = []

def ideal_spectrum(peptide):
    res = []
    tmp_l, tmp_r = 0, 0
    res.append(0)
    for i in range(len(peptide)):
        tmp_l += weights[peptide[i]]
        res.append(tmp_l)
        tmp_r += weights[peptide[len(peptide) - 1 - i]]
        res.append(tmp_r)
    res = list(set(res))
    res.sort()
    return res

def create_graph(spectrum):
    edges = [[] for v in range(len(spectrum))]
    for i in range(len(spectrum)):
        for j in range(i, len(spectrum)):
            if (spectrum[j] - spectrum[i]) in masses:
                edges[i].append((j, acid_from_mass[spectrum[j] - spectrum[i]]))
    return edges

def dfs(v):
    global possible_paths, path
    
    if not edges[v] and ideal_spectrum(''.join(map(str, path))) == spectrum:
        #print(ideal_spectrum(''.join(map(str, path))))
        possible_paths.append(''.join(map(str, path)))
        return
    for u, acid in edges[v]:
        path.append(acid)
        dfs(u)
        path.pop()

import sys

def main():
    global spectrum, edges, possible_paths, path
    sys.stdin = open("test_in.txt", "r")
    sys.stdout = open("test_out.txt", "w")
    input = sys.stdin.readline
    
    spectrum = list(map(int, input().split()))
    
    edges = create_graph(spectrum)
    dfs(0)
    print(possible_paths)
main()

In [96]:
alphabet = [] 
masses = []
weights = dict()
acid_from_mass = dict()

def get_AA_letters_masses():
    letters = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    masses = [71, 103, 115, 129, 147, 57, 137, 113, 128, 113, 131, 114, 97, 128, 156, 87, 101, 99, 186, 163]
    return letters, masses

def get_XZ_letters_masses():
    letters = ['X', 'Z']
    masses = [4, 5]
    return letters, masses

def peptide_vector(peptide):
    pep_vec = []
    tmp, mass = 0, 0
    for acid in peptide:
        mass += weights[acid]
        while tmp < mass - 1:
            pep_vec.append(0)
            tmp += 1
        pep_vec.append(1)
        tmp += 1
    return pep_vec

def score(spec_vec, pep_vec):
    return sum(pep_vec[i] * spec_vec[i] for i in range(len(spec_vec)))

def find_possible_peptides(a, m):
    possible_peptides = []
    l, r, cur = 0, 0, 0
    n = len(a)
    while r < n:
        if cur + weights[a[r]] <= m:
            cur += weights[a[r]]
            if cur == m:
                possible_peptides.append(a[l:r + 1])
            r += 1
        else:
            if r - l > 0:
                cur -= weights[a[l]]
                l += 1
            else:
                r += 1
                l = r
    return list(set(possible_peptides))

def find_best_peptide(spectrum_vector, possible_peptides):
    scores = [score(spectrum_vector, peptide_vector(peptide)) for peptide in possible_peptides]
    index_max, score_max = 0, scores[0]
    for index, elem in enumerate(scores):
        if elem > score_max:
            index_max, score_max = index, elem
    return possible_peptides[index_max]
    
import sys

def main():
    global letters, masses, weights, acid_from_mass
    sys.stdin = open("test_in.txt", "r")
    sys.stdout = open("test_out.txt", "w")
    input = sys.stdin.readline
    
    spectrum_vector = list(map(int, input().split()))
    m = len(spectrum_vector)
    proteome = input().rstrip('\n')
    
    if len(spectrum_vector) > 30: # ключевое место для различия между "тестовым примером" и "реальным"
        letters, masses = get_AA_letters_masses()
        weights = dict(zip(letters, masses))
        acid_from_mass = dict(zip(masses, letters))
    else:
        letters, masses = get_XZ_letters_masses()
        weights = dict(zip(letters, masses))
        acid_from_mass = dict(zip(masses, letters))
    
    possible_peptides = find_possible_peptides(proteome, m)
    print(find_best_peptide(spectrum_vector, possible_peptides))
main()